In [1]:
#установим библиотеки для работы с моделью 
# !pip install git+https://github.com/rcmalli/keras-vggface.git
# !pip install keras_vggface
# !pip install keras_applications
# !pip install  image
# !pip install q keras==2.2.4
# conda install -c conda-forge keras-applications 
# pip install keras-vggface

In [2]:
#импортируем библиотеки
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pickle
import cv2
import numpy as np
from PIL import Image as image_n

In [3]:
#загрузим наш словарь полученный при подготовки данных перед обучением модели 
#для дальнейшей расшифроки эмоций в текстовом виде
with open('./model/class_dict.pickle', 'rb') as f:
    class_dict = pickle.load(f)

In [4]:
#посмотрим на классы
class_dict

{0: 'anger',
 1: 'contempt',
 2: 'disgust',
 3: 'fear',
 4: 'happy',
 5: 'neutral',
 6: 'sad',
 7: 'surprise',
 8: 'uncertain'}

In [5]:
class model_emotions:
    """
    Класс который при инициализации загружают обученную модель 
    Функция prepare_image подготавливает картинку для подачи в нейросеть
    Функция predict_emotions делает прогноз и переводит эмоцию в текст 
    """
    def __init__(self):
        self.model = tf.keras.models.load_model('./model/checkpoint_best.hdf5')
    
    def prepare_image(self,image):
        #меняем размер картинки
        resized_image = cv2.resize(image, (224, 224))
        #разворачиваем массив
        img = np.expand_dims(resized_image, axis=0) 
        #меняем формат
        img = img.astype('float64')
        img = utils.preprocess_input(img, version=2) # мы используем resnet50 -- поэтому version2
        return img
    
    def predict_emotion(self,image):
        predicted = self.model.predict(image)
        predicted_emotion = class_dict.get(np.argmax(predicted))
        return predicted_emotion
        

In [6]:
#создадим класс модели
model = model_emotions()

In [7]:
#подключимся к камере
cam = cv2.VideoCapture(0)

In [8]:
def cutting_out_the_face(img,faces):
    """
    Функция для вырезания лица из картинки
    Перевода из формата BGR в RGB
    Далее вызываем подготовки картинки для модели и прогноза
    Функция возвращает класс эмоции
    """
    #переводим картинку в RGP в формат
    rgb_frame = img[:, :, ::-1]
    one_face = faces[0]
    x, y, w, h = one_face
    face_boundingbox_bgr = img[y:y + h, x:x + w]
    face_boundingbox_rgb = cv2.cvtColor(face_boundingbox_bgr, cv2.COLOR_BGR2RGB)
    #подготавливаем картинку
    image_to_model = model.prepare_image(face_boundingbox_rgb)
    #получаем прогноз
    cls_predicted = model.predict_emotion(image_to_model)
    return cls_predicted


In [9]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#здесь мы в бесконечном цыкле пока пользователь не нажмет (esc) считываем кадры с камеры
while True:
    #получим изображение с камеры
    ret,img = cam.read()
    #загрузим детектор лиц
    faces = face_detector.detectMultiScale(img,scaleFactor=1.5,minNeighbors=5,minSize=(20,20))

    for (x, y, w, h) in faces:
        #добавим boundingbox
        cv2.rectangle(img,(x, y), (x+w, y+h),(0,255,0),3)
        #вызываем функцию для получения эмоции
        emo = cutting_out_the_face(img,faces)
        #вставим текст полученной эмоции
        cv2.putText(img, emo, (y + 150, x - 140), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0),2)
    #визуализируем картинку
    cv2.imshow('From Camera', img)
    #если нажали кнопку esc, то цикл прерывается
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
#закрываем все окна
cam.release()
cv2.destroyAllWindows()

In [10]:
# cam = cv2.VideoCapture(0)
# # cam.set(cv2.CAP_PROP_FRAME_WIDTH, 240)  # ширина кадра -- 640 пикселей
# # cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)  # высота кадра -- 480 пикселей
# def emotions(img,faces):
#     rgb_frame = img[:, :, ::-1]
#     one_face = faces[0]
#     x, y, w, h = one_face
#     face_boundingbox_bgr = img[y:y + h, x:x + w]
#     face_boundingbox_rgb = cv2.cvtColor(face_boundingbox_bgr, cv2.COLOR_BGR2RGB)
#     im = image_n.fromarray(face_boundingbox_rgb)
#     im.save("temp.jpeg")
#     #print(type(face_boundingbox_rgb))
# #     img_1 = image.load_img('temp.jpeg', target_size=(224, 224))
# #     x_1 = image.img_to_array(img_1) #переводим картинку в массив
# #     x_1 = np.expand_dims(x_1, axis=0) #разворачиваем массив
# #     x_1 = utils.preprocess_input(x_1, version=2) # мы используем resnet50 -- поэтому version2
# #     pred = model_load.predict(x_1)
# #     cls = class_dict.get(np.argmax(pred))
#     return cls
# #     plt.imshow(x_1)
# #     plt.show()

# face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# while True:
#     ret,img = cam.read()

#     faces = face_detector.detectMultiScale(img,scaleFactor=1.5,minNeighbors=5,minSize=(20,20))

#     for (x, y, w, h) in faces:
#         cv2.rectangle(img,(x, y), (x+w, y+h),(0,255,0),3)
#         #cv2.rectangle(img, (y, x), (y + h, x + w), (0, 255, 0), 3)
#         emo = emotions(img,faces)
#         cv2.putText(img, emo, (y + 250, x - 140), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0),2)

#     cv2.imshow('From Camera', img)

#     k = cv2.waitKey(30) & 0xFF
#     if k == 27:
#         break

# cam.release()
# cv2.destroyAllWindows()